In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/intents.csv")

df = df.drop_duplicates(subset="text")
df.to_csv("../data/intents_dedup.csv", index=False)

label2id = {"none": 0, "info": 1}
df["label_id"] = df["label"].map(label2id)

df = df.dropna(subset=["label_id"])

print("Tổng số mẫu:", len(df))
display(df.head())

train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["label_id"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label_id"], random_state=42)

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

print("Trùng giữa train và val:", len(set(train_df["text"]) & set(val_df["text"])))
print("Trùng giữa train và test:", len(set(train_df["text"]) & set(test_df["text"])))
print("Trùng giữa val và test:", len(set(val_df["text"]) & set(test_df["text"])))

Tổng số mẫu: 3054


,text,label,label_id
0,Homestay ở Thái Bình có phục vụ ăn sáng không?,info,1
1,Đi Pleiku vào cuối tuần có kẹt xe không?,info,1
2,Sơn Tùng M-TP ra bài mới chưa?,none,0
3,Nên đi Đảo Cát Bà vào mùa nào là đẹp nhất?,info,1
4,Có chuyến bay nào thẳng đến Ninh Thuận không?,info,1


Train: 2137 | Val: 458 | Test: 459
Trùng giữa train và val: 0
Trùng giữa train và test: 0
Trùng giữa val và test: 0


In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "vinai/phobert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import torch
from torch.utils.data import Dataset

class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

# Khởi tạo Dataset
train_dataset = IntentDataset(train_df["text"].tolist(), train_df["label_id"].tolist(), tokenizer)
val_dataset   = IntentDataset(val_df["text"].tolist(), val_df["label_id"].tolist(), tokenizer)
test_dataset  = IntentDataset(test_df["text"].tolist(), test_df["label_id"].tolist(), tokenizer)

In [29]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)

num_epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * num_epochs
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

best_val_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    # Evaluate validation
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            batch = {k:v.to(device) for k,v in batch.items()}
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=-1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(batch["labels"].cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch+1} | Validation accuracy: {val_acc:.4f}")

Epoch 1: 100%|██████████| 134/134 [04:44<00:00,  2.12s/it]


Epoch 1 | Validation accuracy: 0.9891


Epoch 2: 100%|██████████| 134/134 [05:12<00:00,  2.34s/it]


Epoch 2 | Validation accuracy: 0.9978


Epoch 3: 100%|██████████| 134/134 [05:07<00:00,  2.29s/it]


Epoch 3 | Validation accuracy: 0.9978


In [30]:
from sklearn.metrics import classification_report

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
model.eval()
preds, true = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        pred = torch.argmax(outputs.logits, dim=-1)
        preds.extend(pred.cpu().numpy())
        true.extend(batch["labels"].cpu().numpy())

print("Test Accuracy:", accuracy_score(true, preds)) 
print(classification_report(true, preds, target_names=["none", "info"]))

Testing: 100%|██████████| 29/29 [00:12<00:00,  2.27it/s]

Test Accuracy: 1.0
              precision    recall  f1-score   support

        none       1.00      1.00      1.00        11
        info       1.00      1.00      1.00       448

    accuracy                           1.00       459
   macro avg       1.00      1.00      1.00       459
weighted avg       1.00      1.00      1.00       459



In [33]:
def predict_intent(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=-1).cpu().item()
    for k, v in label2id.items():
        if v == pred:
            return k

print(predict_intent("Homestay ở Thái Bình có phục vụ ăn sáng không?"))
print(predict_intent("Sơn Tùng M-TP ra bài mới chưa?"))

pred_df = pd.DataFrame({
    "text": test_df["text"].tolist(),
    "true_label": test_df["label"].tolist(),
    "pred_label": ["info" if p == 1 else "none" for p in preds]
})

output_path = "../data/processed/predict_intents.csv"
pred_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Đã lưu kết quả dự đoán vào: {output_path}")
display(pred_df.head())

info
none
Đã lưu kết quả dự đoán vào: ../data/processed/predict_intents.csv


,text,true_label,pred_label
0,Bạn có biết chỗ nghỉ nào view đẹp ở Chợ Bến Th...,info,info
1,Nếu tôi đến Eo Gió thì nên đi đâu đầu tiên?,info,info
2,"Tôi muốn đi phượt Đảo Bình Ba, có nguy hiểm kh...",info,info
3,Tôi có thể đổi tiền ở đâu khi đến Cần Thơ?,info,info
4,Có chuyến bay nào thẳng đến Mộc Châu không?,info,info


In [32]:
model.save_pretrained("../models/intent_model")
tokenizer.save_pretrained("../models/intent_model")
print("Mô hình và tokenizer đã được lưu thành công.")

Mô hình và tokenizer đã được lưu thành công.
